In [76]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [77]:
city_input = input("City: ")

City: Mexico City


In [78]:
city = city_input.replace(' ', '-').lower()

In [80]:
# URL of page to be scraped
url = 'https://www.cntraveler.com/gallery/best-restaurants-in-{0}'.format(city)

In [81]:
response = requests.get(url)

In [82]:
soup = bs(response.text,'html.parser')

In [84]:
results = soup.find_all('div', class_="image-content-container")

In [88]:
restaurant_list = []
price_list = []
blurb_list = []

for result in results:
    try:
        restaurant = result.find('h2', class_='hed').text
        price = result.find('p', class_='price').text
        blurb = result.find(class_='dek').text
        if (restaurant and price and blurb):
            restaurant_list.append(restaurant)
            price_list.append(price)
            blurb_list.append(blurb)
    except AttributeError as e:
        print(e)

In [95]:
restaurants = pd.DataFrame(list(zip(restaurant_list, price_list, blurb_list)), 
               columns =['Restaurant', 'Price', 'Description'])

,Restaurant,Price,Description
0,Pujol,$$$,Chef Enrique Olvera—the celebrity chef behind ...
1,Quintonil,$$$,There are the Pujol fans and then there are th...
2,Panadería Rosetta,$,There are two locations of this lauded bakery ...
3,Máximo Bistrot,$$$,Máximo Bistrot is a true farm-to-table restaur...
4,La Docena,$$$,"Save for the shrimp po'boy, it's hard to see w..."
5,Nicos,$$$,Nico's is a serious restaurant without any pre...
6,Bar El Sella,$$,"Though technically a bar, Sella is more of an ..."
7,El Hidalguense,$$,The barbacoa is excellent here. Sheep and goat...
8,Lorea,$$$,A demure plaque outside a nondescript Roma tow...
9,Loup Bar,$$,"From Gaetan Rousset and Joaquín Cardoso, Loup ..."
